In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
dialogs = []
with open('human_chat.txt', 'r') as w:
    for line in w:
        try:
            dialogs.append(line.split(': ')[1].split("\n")[0])
        except:
            print('done')
        
len(dialogs)


In [1]:
mh = pd.read_csv('Mental_Health_FAQ.csv')
mh.head()

NameError: name 'pd' is not defined

In [2]:
# data = list(zip(mh.Questions, mh.Answers))
# qna = []
# for i in data:
#     data1 = []
#     data1.append(i[0])
#     data1.append(i[1])
#     qna.append(data1)
    
# print(list(mh.Answers))

NameError: name 'mh' is not defined

In [ ]:
mh.

In [ ]:
ed = pd.read_csv('emotion-emotion_69k.csv')
ed.head(2)

In [ ]:
ed.columns

In [ ]:
emapthetic_dialogs = []
for i in ed.empathetic_dialogues:
    try: 
        emapthetic_dialogs.append(i.split('\nAgent :')[0].split('Customer :')[1])
    except:
        continue
    
len(emapthetic_dialogs)
emapthetic_dialogs

In [ ]:
ed.head(2)

In [ ]:
tc = pd.read_csv('topical_chat.csv')
tc.head()

In [ ]:
tc_dialogs = list(tc.message)

In [ ]:
len(tc_dialogs)

In [ ]:
!pip install cleantext

In [ ]:
import cleantext



def clean(textdata, depth):
    cleaned_text = []
    if depth == 1:
        for i in textdata:
            cleaned_text.append(cleantext.clean(str(i), extra_spaces=True, lowercase=True, stopwords=True, stemming=True, numbers=True, punct=True, clean_all = True))
        
        return cleaned_text
    else:
        for i in textdata:
            for j in i:
                cleaned_text.append(cleantext.clean(str(j), extra_spaces=True, lowercase=True, stopwords=True, stemming=True, numbers=True, punct=True, clean_all = True))
        
        return cleaned_text

In [ ]:
# dialogs[::2] = clean(dialogs[::2], depth = 1)
dialogs[1::2] = clean(dialogs[1::2], depth = 1)

emapthetic_dialogs =clean(emapthetic_dialogs, depth = 1)



tc_dialogs = clean(tc_dialogs, depth = 1)
dialogs[1::2]
# clean(dialogs[1::2], depth = 1)

In [ ]:
all_questions = list(dialogs[::2]) + list(mh.Questions) + list(emapthetic_dialogs[::2]) + list(tc_dialogs[::2])
all_answers = list(dialogs[1::2]) + list(mh.Answers) + list(emapthetic_dialogs[1::2]) + list(tc_dialogs[1::2]) 

In [ ]:
del dialogs

In [ ]:
del mh

In [ ]:
del emapthetic_dialogs

In [ ]:
del tc_dialogs

In [ ]:
len(all_questions)

In [ ]:
len(all_answers)

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing , utils

In [ ]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([str(i) for i in all_questions[:1000]] +  [str(i) for i in all_answers[:1000]])
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

In [ ]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append( word )

def tokenize( sentences ):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub( '[^a-zA-Z]', ' ', sentence )
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append( tokens )
    return tokens_list , vocabulary

#p = tokenize( questions + answers )
#model = Word2Vec( p[ 0 ] ) 

#embedding_matrix = np.zeros( ( VOCAB_SIZE , 100 ) )
#for i in range( len( tokenizer.word_index ) ):
    #embedding_matrix[ i ] = model[ vocab[i] ]

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( [str(i) for i in all_questions[:1000]] )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

In [ ]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( [str(i) for i in all_answers[:1000]] )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )


In [ ]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( [str(i) for i in all_answers[:1000]] )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )



In [ ]:
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

In [ ]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

In [ ]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=150 ) 